In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import probnum as pn
from probnum.problems.zoo.linalg import random_spd_matrix
import scipy.sparse

import probnum_galerkin

In [ ]:
%matplotlib inline

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("pdf", "svg")

In [ ]:
rng = np.random.default_rng(1)

In [ ]:
A = random_spd_matrix(rng, dim=2, spectrum=[1.0, 0.1])
x = np.array([1.0, 1.0])
b = A @ x

In [ ]:
linsys = pn.problems.LinearSystem(A, b, x)

In [ ]:
linsys.solution

In [ ]:
obj_fn = lambda x: 0.5 * probnum_galerkin.linalg.euclidean_inprod(x, x, A) - probnum_galerkin.linalg.euclidean_inprod(b, x)

In [ ]:
xs_plot, ys_plot = np.meshgrid(
    np.linspace(-0.25, 1.2),
    np.linspace(-0.25, 1.2),
)

plt.contour(
    xs_plot, ys_plot,
    obj_fn(np.stack((xs_plot, ys_plot), axis=-1))
)
plt.scatter(
    x[0], x[1],
    marker="*",
    s=100,
)

In [ ]:
solver = probnum_galerkin.linalg.solvers.ProbabilisticLinearSolver(
    probnum_galerkin.linalg.solvers.beliefs.GaussianSolutionBelief.from_linear_system(linsys),
    policy=probnum_galerkin.linalg.solvers.policies.CovariancePolicy(),
    observation_op=probnum_galerkin.linalg.solvers.observation_ops.ResidualMatVec(),
    belief_update=probnum_galerkin.linalg.solvers.belief_updates.GaussianInferenceBeliefUpdate(noise_var=0.1),
    stopping_criteria=(
        probnum_galerkin.linalg.solvers.stopping_criteria.MaxIterations(10),
        probnum_galerkin.linalg.solvers.stopping_criteria.ResidualNorm(),
    ),
)

means = []

for belief, state, done in solver.solve_iter(linsys):
    means.append(belief.x.mean.copy())

In [ ]:
belief.x.mean, belief.x.var

In [ ]:
means

In [ ]:
xs_plot, ys_plot = np.meshgrid(
    np.linspace(-0.25, 1.2),
    np.linspace(-0.25, 1.2),
)

plt.contour(
    xs_plot, ys_plot,
    obj_fn(np.stack((xs_plot, ys_plot), axis=-1))
)
plt.scatter(
    x[0], x[1],
    marker="*",
    s=100,
)
plt.plot(
    tuple(mean[0] for mean in means),
    tuple(mean[1] for mean in means),
)